## API Time Selectors - Using Start, End, and Reftime with Forecast Data

Forecast datasets typically have two time dimensions, `reftime` and `time`. The `reftime` of a forecast data product represents the run time of the forecast model, while `time` represents the actual time (typically in the future) that the values are valid. For each `reftime`, values are unique for each unique `time` value.

Typically the most recent forecast is most desirable to users, and the Planet OS API intentionally returns these values by default. However, there may be cases when one wants to investigate a historical forecast, or how values at a specific time differ between successive forecasts.

The [RTOFS data product](http://data.planetos.com/datasets/noaa_rtofs_surface_1h_diag) is a daily updated forecast that provides a 3-day, hourly forecast. This notebook highlights ways to use the start, end, reftime_start, and reftime_end API parameters and explain the expected response.

In [1]:
%matplotlib inline

import pandas as pd
from shapely.geometry import Point
import simplejson as json
from urllib.parse import urlencode
from urllib.request import urlopen, Request

A Planet OS API key is required to run this notebook. Keys are displayed in the [account settings](http://data.planetos.com/account/settings/) page on the Planet OS Datahub. If you do not have a Planet OS account, you can [sign up for free](http://data.planetos.com/plans).

In [2]:
apikey = 'ENTER_YOUR_API_KEY_HERE'

## Request the Most Recent Model Run

By default, our API returns the most recent model run. The example below returns all variables for the most recent forecast run (e.g. reftime). At the time of running, this was 2016-11-12T00:00:00. Given the hourly resolution and 3-day extent, we expect 72 unique values to be returned.

In [3]:
# Set the Planet OS API query parameters
count = 100
id = 'noaa_rtofs_surface_1h_diag'
lat=39.37858604638528
lon=-72.57739563685647
time_order = 'desc'

query_dict = {'apikey': apikey,
              'count': count,
              'lon': lon,
              'lat': lat,
              'time_order': time_order,
              }
query = urlencode(query_dict)

api_url = "http://api.planetos.com/v1/datasets/%s/point?%s" % (id, query)

request = Request(api_url)
response = urlopen(request)
response_json = json.loads(response.read())

data = response_json['entries']

# let's flatten the response and create a Pandas dataframe
df = pd.io.json.json_normalize(data)

# then index by time using the axes.time column
pd.to_datetime(df["axes.time"])
df.set_index('axes.time', inplace=True)

print(df.count())
df.head()

axes.latitude                            72
axes.longitude                           72
axes.reftime                             72
classifiers.reference_time               72
context                                  72
data.ice_coverage                        72
data.ice_thickness                       72
data.mixed_layer_thickness               72
data.ssh                                 72
data.surface_boundary_layer_thickness    72
data.u_barotropic_velocity               72
data.v_barotropic_velocity               72
dtype: int64


,axes.latitude,axes.longitude,axes.reftime,classifiers.reference_time,context,data.ice_coverage,data.ice_thickness,data.mixed_layer_thickness,data.ssh,data.surface_boundary_layer_thickness,data.u_barotropic_velocity,data.v_barotropic_velocity
axes.time,,,,,,,,,,,,
2016-11-16T00:00:00,39.390355,-72.616137,2016-11-13T00:00:00,2016-11-13T00:00:00,reference_time_MT_lat_lon,0.0,0.0,65.745354,-0.538301,41.573853,0.022478,-0.028018
2016-11-15T23:00:00,39.390355,-72.616137,2016-11-13T00:00:00,2016-11-13T00:00:00,reference_time_MT_lat_lon,0.0,0.0,65.913673,-0.536145,41.650883,0.024630,-0.034462
2016-11-15T22:00:00,39.390355,-72.616137,2016-11-13T00:00:00,2016-11-13T00:00:00,reference_time_MT_lat_lon,0.0,0.0,65.955826,-0.535941,39.685669,0.025145,-0.038177
2016-11-15T21:00:00,39.390355,-72.616137,2016-11-13T00:00:00,2016-11-13T00:00:00,reference_time_MT_lat_lon,0.0,0.0,65.768364,-0.538244,36.706154,0.024137,-0.039219
2016-11-15T20:00:00,39.390355,-72.616137,2016-11-13T00:00:00,2016-11-13T00:00:00,reference_time_MT_lat_lon,0.0,0.0,65.252411,-0.541465,30.857641,0.022561,-0.037685


## Request a Single Model Run

By default, the system will return the most recent model run (e.g. reftime). However, a specific model run can be acquired by requesting an identical reftime_start and reftime_end value. In the example below, we request values from only the 2016-11-12T00:00:00 model run.

In [4]:
# Request only data from a specific model run

reftime_end = '2016-11-12T00:00:00'
reftime_start = '2016-11-12T00:00:00'

query_dict = {'apikey': apikey,
              'count': count,
              'lat': lat,
              'lon': lon,
              'reftime_end': reftime_end,
              'reftime_start': reftime_start,
              'time_order': time_order,
              }
query = urlencode(query_dict)

api_url = "http://api.planetos.com/v1/datasets/%s/point?%s" % (id, query)

request = Request(api_url)
response = urlopen(request)
response_json = json.loads(response.read())

data = response_json['entries']

# let's flatten the response and create a Pandas dataframe
df = pd.io.json.json_normalize(data)

# then index by time using the axes.time column
pd.to_datetime(df["axes.time"])
df.set_index('axes.time', inplace=True)

print(df.count())
df

axes.latitude                            72
axes.longitude                           72
axes.reftime                             72
classifiers.reference_time               72
context                                  72
data.ice_coverage                        72
data.ice_thickness                       72
data.mixed_layer_thickness               72
data.ssh                                 72
data.surface_boundary_layer_thickness    72
data.u_barotropic_velocity               72
data.v_barotropic_velocity               72
dtype: int64


,axes.latitude,axes.longitude,axes.reftime,classifiers.reference_time,context,data.ice_coverage,data.ice_thickness,data.mixed_layer_thickness,data.ssh,data.surface_boundary_layer_thickness,data.u_barotropic_velocity,data.v_barotropic_velocity
axes.time,,,,,,,,,,,,
2016-11-15T00:00:00,39.390355,-72.616137,2016-11-12T00:00:00,2016-11-12T00:00:00,reference_time_MT_lat_lon,0.0,0.0,67.708801,-0.542724,12.434255,0.003629,0.086343
2016-11-14T23:00:00,39.390355,-72.616137,2016-11-12T00:00:00,2016-11-12T00:00:00,reference_time_MT_lat_lon,0.0,0.0,65.582794,-0.542579,10.058778,0.010456,0.085727
2016-11-14T22:00:00,39.390355,-72.616137,2016-11-12T00:00:00,2016-11-12T00:00:00,reference_time_MT_lat_lon,0.0,0.0,63.852642,-0.542803,7.630936,0.016154,0.081681
2016-11-14T21:00:00,39.390355,-72.616137,2016-11-12T00:00:00,2016-11-12T00:00:00,reference_time_MT_lat_lon,0.0,0.0,64.714760,-0.543554,9.274990,0.019874,0.076022
2016-11-14T20:00:00,39.390355,-72.616137,2016-11-12T00:00:00,2016-11-12T00:00:00,reference_time_MT_lat_lon,0.0,0.0,68.936295,-0.545467,18.026541,0.021246,0.070597
2016-11-14T19:00:00,39.390355,-72.616137,2016-11-12T00:00:00,2016-11-12T00:00:00,reference_time_MT_lat_lon,0.0,0.0,69.775475,-0.549083,8.639316,0.020488,0.066938
2016-11-14T18:00:00,39.390355,-72.616137,2016-11-12T00:00:00,2016-11-12T00:00:00,reference_time_MT_lat_lon,0.0,0.0,70.744438,-0.553932,6.574884,0.017690,0.065166
2016-11-14T17:00:00,39.390355,-72.616137,2016-11-12T00:00:00,2016-11-12T00:00:00,reference_time_MT_lat_lon,0.0,0.0,71.715012,-0.559237,6.080428,0.013164,0.064763
2016-11-14T16:00:00,39.390355,-72.616137,2016-11-12T00:00:00,2016-11-12T00:00:00,reference_time_MT_lat_lon,0.0,0.0,72.819054,-0.563363,7.785250,0.007407,0.065613


### All available model run values for a specific time

This example looks for data at a specific future datetime (e.g. 2016-11-14T00:00:00) and returns values from all available model runs (e.g. reftimes). From the result at the time of running, we can see that two forecasts provide values for Nov 14th at 00:00, the 2016-11-11T00:00:00 and 2016-11-12T00:00:00 model runs.

In [5]:
# Request only data from yesterday's model run (e.g. 2016-11-11T00:00:00)

end = '2016-11-14T00:00:00'
start = '2016-11-14T00:00:00'

query_dict = {'apikey': apikey,
              'count': count,
              'end': end,
              'lat': lat,
              'lon': lon,
              'reftime_recent': 'false',
              'start': start,
              'time_order': time_order,
              }
query = urlencode(query_dict)

api_url = "http://api.planetos.com/v1/datasets/%s/point?%s" % (id, query)

request = Request(api_url)
response = urlopen(request)
response_json = json.loads(response.read())

data = response_json['entries']

# let's flatten the response and create a Pandas dataframe
df = pd.io.json.json_normalize(data)

# then index by time using the axes.time column
pd.to_datetime(df["axes.time"])
df.set_index('axes.time', inplace=True)

print(df.count())
df.head()

axes.latitude                            2
axes.longitude                           2
axes.reftime                             2
classifiers.reference_time               2
context                                  2
data.ice_coverage                        2
data.ice_thickness                       2
data.mixed_layer_thickness               2
data.ssh                                 2
data.surface_boundary_layer_thickness    2
data.u_barotropic_velocity               2
data.v_barotropic_velocity               2
dtype: int64


,axes.latitude,axes.longitude,axes.reftime,classifiers.reference_time,context,data.ice_coverage,data.ice_thickness,data.mixed_layer_thickness,data.ssh,data.surface_boundary_layer_thickness,data.u_barotropic_velocity,data.v_barotropic_velocity
axes.time,,,,,,,,,,,,
2016-11-14T00:00:00,39.390355,-72.616137,2016-11-12T00:00:00,2016-11-12T00:00:00,reference_time_MT_lat_lon,0.0,0.0,76.085350,-0.580702,56.848572,-0.017093,0.053446
2016-11-14T00:00:00,39.390355,-72.616137,2016-11-13T00:00:00,2016-11-13T00:00:00,reference_time_MT_lat_lon,0.0,0.0,77.897614,-0.583224,43.537800,-0.024332,0.050457
